In [1]:
import glob
import os
import json
import pickle
import yaml

import argparse
import json
import os
import pathlib
import pickle as pkl
import shutil
import sys
import time
import warnings
from collections import defaultdict
from typing import List, Optional, Union

import awkward as ak
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import pyarrow as pa
import pyarrow.parquet as pq

import uproot
from coffea import nanoevents, processor
from coffea.analysis_tools import PackedSelection, Weights
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory
from coffea.nanoevents.methods import candidate, vector
from coffea.nanoevents.methods.nanoaod import FatJetArray, GenParticleArray

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.CMS)

import sys
sys.path.append("../boostedhiggs/")
import btag

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
! ls ../rootfiles/

ggh.root


In [4]:
### schema
nanoevents.PFNanoAODSchema.mixins["PFCands"] = "PFCand"
nanoevents.PFNanoAODSchema.mixins["SV"] = "PFCand"

# load a ggF root file into coffea-friendly NanoAOD structure
import uproot
f = uproot.open(f"../rootfiles/ggh.root")
num = f['Events'].num_entries   ### checks number of events per file 
print(f'number of events per file: {num}')

events = nanoevents.NanoEventsFactory.from_root(f, "Events", schemaclass=nanoevents.PFNanoAODSchema).events()

number of events per file: 6000


/uscms/home/fmokhtar/nobackup/miniconda3/envs/coffea-env/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (2140 branches) at 0x7f6c34144580>, taking first instance
  warnings.warn(
/uscms/home/fmokhtar/nobackup/miniconda3/envs/coffea-env/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (2140 branches) at 0x7f6c34144580>, taking first instance
  warnings.warn(
/uscms/home/fmokhtar/nobackup/miniconda3/envs/coffea-env/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (2140 branches) at 0x7f6c34144580>, taking first instance
  warnings.warn(
/uscms/home/fmokhtar/nobackup/miniconda3/envs/coffea-env/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch

In [5]:
nevents = len(events)
isMC = hasattr(events, "genWeight")
weights = Weights(nevents, storeIndividual=True)

weights.weight()

array([1., 1., 1., ..., 1., 1., 1.])

In [6]:
# from hww processor
fatjets = events.FatJet
good_fatjets = (fatjets.pt > 200) & (abs(fatjets.eta) < 2.5) & fatjets.isTight
good_fatjets = fatjets[good_fatjets]  # select good fatjets
candidatefj = ak.firsts(good_fatjets)

# apply btag weight for candidatefj
btag.add_btag_weights(weights, "2018", ak.singletons(candidatefj), "T")

weights.weight()

/uscms/home/fmokhtar/nobackup/miniconda3/envs/coffea-env/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in divide
  result = getattr(ufunc, method)(


array([1.04048248, 1.05878781, 1.05162773, ..., 1.0217171 , 1.03197611,
       0.98962473])